<a href="https://colab.research.google.com/github/ahmadsaad007/Deep_Learning_Projects/blob/master/Predicting_Eye_Diseases_From_Fundus_Photographs_Using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import *
from keras.layers.pooling import MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K

In [0]:
import PIL
from PIL import Image
import keras
from numpy import asarray
from os import listdir
from matplotlib import image

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
train_path = "C:\\Users\\Ahmad Saad\\Desktop\\Research\\train"
test_path = "C:\\Users\\Ahmad Saad\\Desktop\\Research\\test"

In [6]:
imgdata = ImageDataGenerator(validation_split=0.3, rescale=1.0/255.0)
train_batches = imgdata.flow_from_directory(train_path,subset='training', target_size=(512,496) ,
                                            classes = ['CNV','NORMAL'], batch_size = 32)
test_batches = imgdata.flow_from_directory(test_path, target_size=(512,496) ,
                                           classes = ['CNV','NORMAL'], batch_size = 16)
valid_batches = imgdata.flow_from_directory(train_path, subset='validation', target_size=(512,496) ,
                                            classes = ['CNV','NORMAL'], batch_size = 32)

model = Sequential()
model.add(Conv2D(32, (3,3),activation='relu', padding='same', input_shape=(512,496,3)))
model.add(Conv2D(32, (3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3,3),activation='relu', padding='valid'))
model.add(Conv2D(64, (3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
#model.add(Dense(128, activation ='relu'))
#model.add(Dense(256, activation ='relu'))
model.add(Dense(32, activation ='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation ='relu'))
model.add(Dense(2, activation='softmax'))

model.summary()

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 512, 496, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 510, 494, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 255, 247, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 255, 247, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 253, 245, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 251, 243, 64)      36928     
___

# New Section

In [0]:
from sklearn.utils import class_weight

In [0]:
mapp = {}
for e in train_batches.classes:
    if(e not in mapp):
        mapp[e]=1
    else:
        value = mapp[e]
        mapp[e] = value+1
print(mapp)

{0: 26044, 1: 18421}


In [0]:
for ele in mapp:
    print("Value: %d is that times:" % ele, mapp[ele]/8616)

Value: 0 is that times: 3.022748375116063
Value: 1 is that times: 2.1379990714948933


In [0]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_batches.classes), train_batches.classes)

In [0]:
print(class_weights)

[0.85365151 1.20691059]


In [0]:
model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit_generator(train_batches, steps_per_epoch=8, epochs=12,verbose=1, class_weight=class_weights, 
                    validation_data=valid_batches, validation_steps=8, shuffle=True)

Epoch 1/12
8/8 [==============================] - 333s 42s/step - loss: 6.4221 - acc: 0.6016 - val_loss: 7.1776 - val_acc: 0.5547
Epoch 2/12
8/8 [==============================] - 313s 39s/step - loss: 6.7998 - acc: 0.5781 - val_loss: 6.7369 - val_acc: 0.5820
Epoch 3/12
8/8 [==============================] - 255s 32s/step - loss: 6.9887 - acc: 0.5664 - val_loss: 7.5554 - val_acc: 0.5312
Epoch 4/12
8/8 [==============================] - 236s 30s/step - loss: 6.6109 - acc: 0.5898 - val_loss: 6.8628 - val_acc: 0.5742
Epoch 5/12
8/8 [==============================] - 272s 34s/step - loss: 8.3109 - acc: 0.4844 - val_loss: 7.1146 - val_acc: 0.5586
Epoch 6/12
8/8 [==============================] - 353s 44s/step - loss: 6.5480 - acc: 0.5938 - val_loss: 7.1776 - val_acc: 0.5547
Epoch 7/12
8/8 [==============================] - 248s 31s/step - loss: 6.8628 - acc: 0.5742 - val_loss: 7.0517 - val_acc: 0.5625
Epoch 8/12
8/8 [==============================] - 310s 39s/step - loss: 6.6109 - acc: 0.58

KeyboardInterrupt: 

In [0]:
acc = model.evaluate_generator(test_batches, steps=len(test_batches), verbose=1)


KeyboardInterrupt: 

In [0]:
print((acc[1] * 100))

49.8
